# Data Processing

In [1]:
import pandas as pd
import numpy as np
import os

In [ ]:
cwd = os.getcwd()
cwd

In [3]:
aspects = ["Heart", "Grail", "Forge", "Lantern", "Scale", "Edge", "Nectar", "Winter", "Sky", "Moth", "Knock", "Rose", "Moon"]
wisdoms = ["Illumination", "Hushery", "Nyctodromy", "Skolekosophy", "The Bosk", "Preservation", "Birdsong", "Horomachistry", "Ithastry"]

stations = pd.read_csv("../data/stations.csv")
skills = pd.read_csv("../data/skills.csv")
soul_pieces = pd.read_csv("../data/soul_pieces.csv")
memories = pd.read_csv("../data/memories.csv")

soul_pieces["Aspects"] = ""

for aspect in aspects:
    stations[aspect] = 0
    soul_pieces[aspect] = 0
    memories[aspect] = 0

for wisdom in wisdoms:
    skills[wisdom] = ""

skills[skills.isna()] = 0

## Stations

In [4]:
for row_index, row_data in stations.iterrows():
    for aspect in aspects:

        if aspect in row_data["Allowed Principles"]:
            stations.loc[row_index, aspect] = 1

# Soul Pieces

In [ ]:
for row_index, row_data in soul_pieces.iterrows():
    for aspect in aspects:
        soul_aspects_list = []
        soul_aspects = row_data["Principles"].split(", ")
        for soul_aspect in soul_aspects:
            name, value = soul_aspect.split(" ")
            soul_pieces.loc[row_index, name] = int(value)
            soul_aspects_list.append(name)
        soul_pieces.loc[row_index, "Aspects"] = ", ".join(soul_aspects_list)


soul_pieces_list = list(soul_pieces["Element"].values)
soul_pieces_list

## Memories

In [6]:
for row_index, row_data in memories.iterrows():
    for aspect in aspects:
        soul_aspects = row_data["Aspect"].split(", ")
        for soul_aspect in soul_aspects:
            value, name = soul_aspect.split(" ")
            memories.loc[row_index, name] = int(value)

## Skills

In [7]:
for row_index, row_data in skills.iterrows():
    for col in ["Tree of Wisdoms Lore 1", "Tree of Wisdoms Lore 2"]:
        wisdom = row_data[col].split(") ")[0]
        wisdom, soul_piece = wisdom.split(", ")
        wisdom = wisdom[1:]

        skill_aspects = row_data[aspects]
        skill_aspects = list(skill_aspects[skill_aspects>0].index)

        soul_aspects = soul_pieces[soul_pieces["Element"] == soul_piece]["Aspects"].values[0].split(", ")

        station_list = []
        wisdom_stations = stations[stations["Wisdom"] == wisdom]
        for station_index, station_data in wisdom_stations.iterrows():
            if sum(station_data[skill_aspects].values) > 0 and sum(station_data[soul_aspects].values)>0:
                station_list.append(station_data["Workstation"])
        # print(station_list)

        wisdom_memories = memories[memories["Evolve via"] == wisdom]["Memory"].values
        # print(wisdom_memories)

        skills.loc[row_index, wisdom] = soul_piece + ", Altars: " + str(station_list) + ", Memories: " + str(wisdom_memories)



In [9]:
skills.to_csv("../data/skills_calculated.csv")